 LOADING DATA INTO BIG QUERY

In [1]:
import mysql.connector as connection
import pandas as pd

def data_pipeline_mysql_to_by(**kwargs):
    mysql_host = kwargs.get('mysql_host')
    mysql_database = kwargs.get('mysql_database')
    mysql_user = kwargs.get('mysql_user')
    mysql_password = kwargs.get('mysql_password')
    bq_project_id = kwargs.get('bq_project_id')
    dataset = kwargs.get('database')

    try: 
        mydb = connection.connect(host=mysql_host, database= mysql_database, user= mysql_user, passwd = mysql_password, use_pure= True)
        all_tables = "Select table_name from information_schema.tables where table_schema = '{}'".format(mysql_database)
        df_tables = pd.read_sql(all_tables, mydb, parse_dates = {'Date':{'format': '%Y-%m-%d'}})

        for table in df_tables.TABLE_NAME:
            table_name = table
            # Extract  table data from MYSQL
            df_table_data = extract_table_from_mysql(table_name, mydb)
            # Transform table data from MYSQL
            df_table_data = transform_data_from_table(df_table_data)
            # Load Data to Big Query
            load_data_into_bigquery(bq_project_id, dataset, table_name, df_table_data)
            # Confirmation message if this is successful
            print("Ingested table{}".format(table_name))
            mydb.close()
    except Exception as e:
            mydb.close()
            print(str(e))


# ETL Proccesses
            
def extract_table_from_mysql(table_name, my_sql_connection):
     # Extract data from table
     extract_query = 'select * from' + table_name
     df_table_data = pd.read_sql(extract_query, my_sql_connection)
     return df_table_data

def transform_data_from_table(df_table_data):
     # Clean dates - convert to string
     object_cols = df_table_data.select.dtypes(include=['object']).columns

     for column in object_cols:
          dtype= str(type(df_table_data[column.valuers[0]]))
          if dtype == "<class 'datetime.date'>":
               df_table_data[column] = df_table_data[column].map(lambda x: str(x)) 
               return df_table_data
          
def load_data_into_bigquery(bq_project_id, dataset, table_name, df_table_data):
     import pandas_gbq as pdbq
     full_table_name_bg = "{}.{}".format(dataset, table_name)
     pdbq.to_gbq(df_table_data, full_table_name_bg, project_id = bq_project_id, if_exists='replace')

SyntaxError: expected ':' (3497165366.py, line 28)

CALL MAIN FUNCTION

In [3]:
kwargs = {
    #BigQuery Connection Details
    'bq_project_id': 'bq_project_id', #CHANGE TO YOUR CREDS
    'dataset': 'dataset_name', #CHANGE TO NAME OF THE DATASET

    #MYSQL Connection details
    'mysql_host': 'mysql_host', #CHANGE TO YOUR CREDS
    'mysql_user': 'mysql_user', #CHANGE TO YOUR CREDS
    'mysql_password': 'mysql_password', #CHANGE TO YOUR CREDS
    'mysql_database': 'mysql_database', #CHANGE TO YOUR CREDS
}

data_pipeline_mysql_to_by(kwargs)